In [2]:
from gurobipy import *
import pandas as pd
import gurobipy as gp

ModuleNotFoundError: No module named 'gurobipy'

In [3]:
distances = pd.read_excel('distances.xlsx', index_col=0)
distances = distances.drop(["Price", "X Position", "Y Position", "Item"], axis = 1)
distances = distances.set_axis([idx for idx in distances.index], axis = 1)
#distances[57] = distances[0]
display(distances)

distances = {(i, j): distances.at[i, j] for i in distances.index for j in list(distances.set_index(distances.columns).reset_index(drop = True).index)}

c = distances


NameError: name 'pd' is not defined

In [4]:
display(c)

NameError: name 'c' is not defined

In [5]:
distances.index

NameError: name 'distances' is not defined

In [6]:
distances.shape[1]

NameError: name 'distances' is not defined

In [7]:
model = gp.Model("Supermarket_Sweep")

distances = pd.read_excel('distances.xlsx', index_col=0)
n=len(distances)-4
times= pd.read_excel('times.xlsx', index_col=0)
prices= pd.read_excel('times.xlsx')
prices=prices["Price"]
x = {}
y = {}
#distances 
print(n)
t = {}
#time
M = 1000 

for a in range(1, 3):
    for i in range(1, n+2):
        for j in range(1, n+2):
            x[a, i, j] = model.addVar(vtype=GRB.BINARY, name=f"x_{a}_{i}_{j}")
            t[a, i, j] = model.addVar(name=f"t_{a}_{i}_{j}")
for a in range(1, 3):
    for j in range(1, n+2):
        y[a, j] = model.addVar(name=f"y_{a}_{i}_{j}")

# Objective Function
model.setObjective(quicksum(prices[j] * x[a, i, j] for i in range(1, n) for j in range(2, n+2) for a in range(1, 3)), GRB.MAXIMIZE)


#constraint adding for part b formulation

model.addConstr(0.1 * y[1, n+1] + quicksum(2 * x[1, i, j] for i in range(1, n+1) for j in range(1, n+2)) <= 22)
model.addConstr(0.1 * y[2, n+1] + quicksum(2 * x[2, i, j] for i in range(1, n+1) for j in range(1, n+2)) <= 22)

    # Constraint 2
#model.addConstr(quicksum(x[a, i, j] for i in range(1, n) for j in range(2, n+2)) <= 10)
model.addConstr(quicksum(x[1, i, j] for i in range(1, n) for j in range(1, n+2)) <= 9)
model.addConstr(quicksum(x[2, i, j] for i in range(1, n) for j in range(1, n+2)) <= 9)
#model.addConstr(quicksum(x[a, i, j] for i in range(1, n+1) for j in range(2, n+2)) <= 10 for a in range(1, 3))
   
    # Constraint 3
model.addConstr(quicksum(x[a, 1, j] + x[a, 2, j] for j in range(2, n+2)) <= 1)
#model.addConstr(quicksum(x[1, i, j] + x[2, i, j] for i in range(1, n+1) for j in range(2, n+2)) <= 1)
   
#sales person constraints 
model.addConstrs(t[a, i, j] <= M * x[a, i, j] for a in range(1, 3) for i in range(1, n) for j in range(2, n+2))
model.addConstrs(y[a, 1] == 0 for a in range(1, 3))

model.addConstrs(y[1, j] <= quicksum(t[1, i, j] for i in range(1, n + 1)) for j in range(2, n + 1) for j in range(1, n + 1))
model.addConstrs(y[2, j] <= quicksum(t[2, i, j] for i in range(1, n + 1)) for j in range(2, n + 1) for j in range(1, n + 1))
model.addConstrs(y[1, j] >= quicksum(t[1, i, j] for i in range(1, n + 1)) for j in range(2, n + 1) for j in range(1, n + 1))
model.addConstrs(y[2, j] >= quicksum(t[2, i, j] for i in range(1, n + 1)) for j in range(2, n + 1) for j in range(1, n + 1))


model.addConstrs(quicksum(t[a, j, k] for k in range(2, n + 1)) == y[a, j] + quicksum(c[j, k] * x[a, j, k]for j in range(i, n+1)) for a in range(1, 3))
#model.addConstrs(quicksum(t[a, j, k]) >= y[a, j] + quicksum(c[j, k] * x[i, j, k]) for i in range(i, n+1) for a in range(1, 3))
#model.addConstrs((x[a, i, j] == 0) or (x[a, i, j] == 1) for a in range(1, 3) for i in range(1, n+1) for j in range(2, n+2))

# Ensure each shopper leaves node one once
model.addConstrs(quicksum(x[1, 1, j]for j in range(2, n + 1)) == 1 for j in range(2, n + 1))
model.addConstrs(quicksum(x[2, 1, j]for j in range(2, n + 1)) == 1 for j in range(2, n + 1))
#model.addConstrs(quicksum(x[1, 1, j]for j in range(2, n + 1)) >= 1 for j in range(2, n + 1))
#model.addConstrs(quicksum(x[2, 1, j]for j in range(2, n + 1)) >= 1 for j in range(2, n + 1))

# Ensure each shopper enters node one once
model.addConstrs(quicksum(x[1, i, 1]for i in range(1, n + 1)) == 1 for i in range(1, n + 1))
model.addConstrs(quicksum(x[2, i, 1]for i in range(1, n + 1)) == 1 for i in range(1, n + 1))
#model.addConstrs(quicksum(x[1, i, 1]for i in range(1, n + 1)) >= 1 for i in range(1, n + 1))
#model.addConstrs(quicksum(x[2, i, 1]for i in range(1, n + 1)) >= 1 for i in range(1, n + 1))

# Shoppers leave and enter other nodes at most once
model.addConstrs(quicksum(x[1, i, j] for j in range(2, n + 1) for i in range(1,n)) <= 1 for i in range (2, n+1))
model.addConstrs(quicksum(x[2, i, j] for j in range(2, n + 1) for i in range(1,n)) <= 1 for i in range (2, n+1))
#model.addConstrs(quicksum(x[1, j, i] for i in range(1, n + 1)) <= 1 for j in range (1, n))
#model.addConstrs(quicksum(x[2, j, i] for i in range(1, n + 1)) <= 1 for j in range (1, n))



# If a shopper enters a node they must leave it
model.addConstrs(quicksum(x[1, i, j] for j in range(1, n)) == quicksum(x[1, j, i] for j in range(1, n)) for i in range (1, n))
#model.addConstrs(quicksum(x[1, i, j] for j in range(1, n + 1)) >= quicksum(x[1, j, i] for j in range(1, n+1)) for i in range (1, n+1))

# We don't use the x[a, i, i] items so that we dont just stay at the same node for no reason
#model.addConstrs(quicksum(x[1, i, i] for i in range (1, n + 1)) < 1)

# if a shopper leaves goes to item j, they must also leave that item

model.optimize()


NameError: name 'gp' is not defined

In [8]:
for var in model.getVars():
    if var.X != 0:
        print(f"{var.VarName}: {var.X}")

NameError: name 'model' is not defined

In [9]:
for var in model.getVars():
    if 'y' in var.VarName:
        print(var.VarName)

NameError: name 'model' is not defined

In [10]:
model.write('TSP.lp')

NameError: name 'model' is not defined

In [11]:
display(t)

NameError: name 't' is not defined

In [12]:
#part e formulation

model.setObjective(quicksum(prices[i] * x[a, i, j] for i in range(1, n+1) for j in range(2, n+2) for a in range(1, 3)), GRB.MAXIMIZE)

#constraint 1 - shopper 1 has no more than one minute
model.addConstr(0.1 * y[n+1] + quicksum(2 * x[i, j] for i in range(1, n) for j in range(2, n+2)) <= 60)

#constraint 2 - basket limit
model.addConstr(quicksum(x[i, j] for i in range(1, n+1) for j in range(2, n+2)) <= 10)
                   
#constraint 3 - no same item twice 
model.addConstrs(quicksum(x[i, j] for i in range(1, n+1)) <= 1 for j in range(2, n+2))

#TSP constraints 
model.addConstrs(t[i, j] <= M * x[i, j] for i in range(1, n+1) for j in range(2, n+2))
model.addConstr(y[1] == 0)
model.addConstrs(y[j] == quicksum(t[i, j] for i in range(1, n+1)) for j in range(1, n+2))
model.addConstrs((x[i, j] == 0) or (x[i, j] == 1) for i in range(1, n+1) for j in range(2, n+2))

#adding the dij constraint
model.addConstrs(d[i] == 0 for i in range(1, n+1) if x[i, 2] == 1)
model.addConstrs(d[i] >= 1 for i in range(1, n+1) if x[i, 2] == 0)

#the rest hehe

model.setObjective(quicksum(prices[i] * x[a, i, j] for i in range(1, n+1) for j in range(2, n+2) for a in range(1, 3)), GRB.MAXIMIZE)


#constraint 1 - shopper 1 has no more than one minute
model.addConstr(0.1 * y[n+1] + quicksum(2 * x[i, j] for i in range(1, n) for j in range(2, n+2)) <= 60)

#constraint 2 - basket limit
model.addConstr(quicksum(x[i, j] for i in range(1, n+1) for j in range(2, n+2)) <= 10)
                   
#constraint 3 - no same item twice 
model.addConstrs(quicksum(x[i, j] for i in range(1, n+1)) <= 1 for j in range(2, n+2))

#TSP constraints 
model.addConstrs(t[i, j] <= M * x[i, j] for i in range(1, n+1) for j in range(2, n+2))
model.addConstr(y[1] == 0)
model.addConstrs(y[j] == quicksum(t[i, j] for i in range(1, n+1)) for j in range(1, n+2))
model.addConstrs((x[i, j] == 0) or (x[i, j] == 1) for i in range(1, n+1) for j in range(2, n+2))




NameError: name 'model' is not defined

In [265]:
help(model)


  Gurobi model object.  Commonly used methods on this object are:
    getConstrs(): Get a list of constraints in the model
    getJSONSolution(): Get a JSON-string representation of the current solution(s) to the model
    getParamInfo(paramname): Get information on a model parameter.
    getVars(): Get a list of variables in the model
    optimize(): Optimize the model.
    printAttr(attrname, filter): Print attribute values.
    printQuality(): Print solution quality statistics.
    printStats(): Print model statistics.
    read(filename): Read model data (MIP start, basis, etc.) from a file
    reset(): Discard any resident solution information.
    resetParams(): Reset all parameters to their default values.
    setParam(paramname, newval): Set a model parameter to a new value.
    write(filename): Write model data to a file.

  Models have a number of attributes that can be queried or modified.
  For example, "print model.objval" prints the objective value of
  the current soluti